In [1]:
import pandas as pd
import numpy as np
from sup_data_to_fasta import load_xlsx, map_raw_data_to_queries_and_expected

In [2]:
esp_df = pd.read_csv('../output/ESP-OD2w11_results.csv').drop(columns=['Unnamed: 0'])

In [3]:
len(esp_df)

36307

In [4]:
total_alignments = len(esp_df[esp_df['qseqid'].notnull()])
not_aligned = len(esp_df[esp_df['qseqid'].isnull()])
print(f"Number of alignments in ESP-OD2w11: {total_alignments}")
print(f"Not aligned in ESP-OD2w11: {not_aligned}")

Number of alignments in ESP-OD2w11: 36307
Not aligned in ESP-OD2w11: 0


In [5]:
len(esp_df[esp_df['sseq_cds'] == esp_df['sseq_genome']])

3726

In [6]:
cds_columns = esp_df.filter(like='_cds')
cds_columns


,qgi_cds,qacc_cds,qaccver_cds,qlen_cds,sseqid_cds,sallseqid_cds,sgi_cds,sallgi_cds,sacc_cds,saccver_cds,...,staxids_cds,sscinames_cds,scomnames_cds,sblastnames_cds,sskingdoms_cds,stitle_cds,salltitles_cds,sstrand_cds,qcovs_cds,qcovhsp_cds
0,0.0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,0.0,0.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,plus,31.0,31.0
1,0.0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212.0,lcl|NC_016810.1_cds_WP_001520210.1_486,lcl|NC_016810.1_cds_WP_001520210.1_486,0.0,0.0,lcl|NC_016810.1_cds_WP_001520210.1_486,lcl|NC_016810.1_cds_WP_001520210.1_486,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001520210.1_486 [gene=h...,lcl|NC_016810.1_cds_WP_001520210.1_486 [gene=h...,plus,13.0,13.0
2,0.0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212.0,lcl|NC_017718.1_cds_WP_000142425.1_4637,lcl|NC_017718.1_cds_WP_000142425.1_4637,0.0,0.0,lcl|NC_017718.1_cds_WP_000142425.1_4637,lcl|NC_017718.1_cds_WP_000142425.1_4637,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_017718.1_cds_WP_000142425.1_4637 [locus...,lcl|NC_017718.1_cds_WP_000142425.1_4637 [locus...,plus,7.0,7.0
3,0.0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212.0,lcl|NC_016810.1_cds_WP_000809815.1_3387,lcl|NC_016810.1_cds_WP_000809815.1_3387,0.0,0.0,lcl|NC_016810.1_cds_WP_000809815.1_3387,lcl|NC_016810.1_cds_WP_000809815.1_3387,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_000809815.1_3387 [gene=...,lcl|NC_016810.1_cds_WP_000809815.1_3387 [gene=...,plus,8.0,8.0
4,0.0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,0.0,0.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,plus,31.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36302,0.0,a63f24c03e5643f2a136bf3425def72f,a63f24c03e5643f2a136bf3425def72f,59.0,lcl|NC_016810.1_cds_WP_001120231.1_4439,lcl|NC_016810.1_cds_WP_001120231.1_4439,0.0,0.0,lcl|NC_016810.1_cds_WP_001120231.1_4439,lcl|NC_016810.1_cds_WP_001120231.1_4439,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001120231.1_4439 [gene=...,lcl|NC_016810.1_cds_WP_001120231.1_4439 [gene=...,plus,22.0,22.0
36303,0.0,a63f24c03e5643f2a136bf3425def72f,a63f24c03e5643f2a136bf3425def72f,59.0,lcl|NC_016810.1_cds_WP_000057014.1_679,lcl|NC_016810.1_cds_WP_000057014.1_679,0.0,0.0,lcl|NC_016810.1_cds_WP_000057014.1_679,lcl|NC_016810.1_cds_WP_000057014.1_679,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_000057014.1_679 [gene=t...,lcl|NC_016810.1_cds_WP_000057014.1_679 [gene=t...,minus,32.0,32.0
36304,0.0,a63f24c03e5643f2a136bf3425def72f,a63f24c03e5643f2a136bf3425def72f,59.0,lcl|NC_016810.1_cds_WP_001526312.1_394,lcl|NC_016810.1_cds_WP_001526312.1_394,0.0,0.0,lcl|NC_016810.1_cds_WP_001526312.1_394,lcl|NC_016810.1_cds_WP_001526312.1_394,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001526312.1_394 [gene=s...,lcl|NC_016810.1_cds_WP_001526312.1_394 [gene=s...,plus,27.0,27.0
36305,0.0,e6b5fe3f467c4f05a700e1f2afae00fe,e6b5fe3f467c4f05a700e1f2afae00fe,51.0,lcl|NC_016810.1_cds_WP_010988985.1_826,lcl|NC_016810.1_cds_WP_010988985.1_826,0.0,0.0,lcl|NC_016810.1_cds_WP_010988985.1_826,lcl|NC_016810.1_cds_WP_010988985.1_826,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_010988985.1_826 [locus_...,lcl|NC_016810.1_cds_WP_010988985.1_826 [locus_...,plus,33.0,33.0


In [7]:
esp_df['stitle_cds'].isnull().sum()

12

In [8]:
esp_df['gene'] = esp_df['stitle_cds'].str.extract(r'gene=(\w+)')
esp_df['locus_tag'] = esp_df['stitle_cds'].str.extract(r'locus_tag=(\w+)')
esp_df['expected_gene'] = esp_df['name'].str.extract(r'(\w+)\(')
esp_df['expected_locus_tag'] = esp_df['name'].str.extract(r'(SL\d+_\d+)')


In [9]:
def was_there_query_not_aligned_w_expected_gene(df):
    grouped = df.query("gene == expected_gene").groupby('query_id')['query_id'].count()
    
    return (grouped == 0).any()

In [29]:
def matches(row):
    start_matches = abs(row['from'] - row['sstart_genome']) <= 20
    end_matches = abs(row['to'] - row['send_genome']) <= 20
    # end_matches = True
    return start_matches and end_matches

esp_genome_df = esp_df[esp_df['sseq_genome'].notnull()]

esp_genome_df['matches'] = esp_genome_df.apply(matches, axis=1)


In [30]:
esp_genome_df.groupby('query_id')['matches'].sum().value_counts()

matches
0    1210
2      17
3       8
4       7
6       4
5       3
1       1
7       1
9       1
Name: count, dtype: int64

In [11]:
was_there_query_not_aligned_w_expected_gene(esp_df)

False

In [12]:
aligned_w_differ_gene = esp_df.query("expected_gene != gene")

In [13]:
locus_tag_second_number = esp_df.expected_locus_tag.str.extract(r'SL\d+_(\d+)')[0].unique()

In [14]:
esp_df

,qseqid,qgi_genome,qacc_genome,qaccver_genome,qlen_genome,sseqid_genome,sallseqid_genome,sgi_genome,sallgi_genome,sacc_genome,...,Fisher's exact test p-value,type,seq,origin,chimera_idx,query_id,gene,locus_tag,expected_gene,expected_locus_tag
0,3787b7f3ccd5425eb0e1b22140b3390a,0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,3787b7f3ccd5425eb0e1b22140b3390a,thrL,SL1344_RS00005,thrL,SL1344_0001
1,3787b7f3ccd5425eb0e1b22140b3390a,0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,3787b7f3ccd5425eb0e1b22140b3390a,htpG,SL1344_RS02480,thrL,SL1344_0001
2,3787b7f3ccd5425eb0e1b22140b3390a,0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,3787b7f3ccd5425eb0e1b22140b3390a,NaN,SL1344_RS24260,thrL,SL1344_0001
3,3787b7f3ccd5425eb0e1b22140b3390a,0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,3787b7f3ccd5425eb0e1b22140b3390a,arcB,SL1344_RS17190,thrL,SL1344_0001
4,3787b7f3ccd5425eb0e1b22140b3390a,0,3787b7f3ccd5425eb0e1b22140b3390a,3787b7f3ccd5425eb0e1b22140b3390a,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,1.890000e-61,CDS,AAAATTTATTGACTTAGGCGGGCAGATACTTTAACCAATATAGGAA...,RNA1,0,3787b7f3ccd5425eb0e1b22140b3390a,thrL,SL1344_RS00005,thrL,SL1344_0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36302,a63f24c03e5643f2a136bf3425def72f,0,a63f24c03e5643f2a136bf3425def72f,a63f24c03e5643f2a136bf3425def72f,59,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,6.810000e-45,sRNA,ATTTCCCTGGTGTTGGCGCAGTATTCGCGCACCCCGGTCAAACCGG...,RNA2,853,a63f24c03e5643f2a136bf3425def72f,tamA,SL1344_RS22630,ArcZ,NaN
36303,a63f24c03e5643f2a136bf3425def72f,0,a63f24c03e5643f2a136bf3425def72f,a63f24c03e5643f2a136bf3425def72f,59,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,6.810000e-45,sRNA,ATTTCCCTGGTGTTGGCGCAGTATTCGCGCACCCCGGTCAAACCGG...,RNA2,853,a63f24c03e5643f2a136bf3425def72f,tcuC,SL1344_RS03480,ArcZ,NaN
36304,a63f24c03e5643f2a136bf3425def72f,0,a63f24c03e5643f2a136bf3425def72f,a63f24c03e5643f2a136bf3425def72f,59,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,6.810000e-45,sRNA,ATTTCCCTGGTGTTGGCGCAGTATTCGCGCACCCCGGTCAAACCGG...,RNA2,853,a63f24c03e5643f2a136bf3425def72f,sbcC,SL1344_RS02020,ArcZ,NaN
36305,e6b5fe3f467c4f05a700e1f2afae00fe,0,e6b5fe3f467c4f05a700e1f2afae00fe,e6b5fe3f467c4f05a700e1f2afae00fe,51,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,9.910000e-45,sRNA,AAAAAGTAACATGGGGCTGTATTTCCCCCTCTGGATGAGGGGGCTT...,RNA2,854,e6b5fe3f467c4f05a700e1f2afae00fe,NaN,SL1344_RS04245,STnc740,NaN
